### Get Local Tourist Destinations

In [ ]:
from bs4 import BeautifulSoup, NavigableString
import requests
from selenium import webdriver

def scrape_tourist_dest_list():
   tourist_dest_url = 'https://www.traveling-up.com/travel-guide-81-provinces-of-the-philippines'

   driver = webdriver.Firefox(executable_path='geckodriver.exe')
   driver.get(tourist_dest_url)
   html = driver.page_source

   data = BeautifulSoup(html,'lxml')
   ul_list= data.find("div",{"class":"entry-content"}).find_all("ul")

   tourist_dest_list = []
   for ul in ul_list:
      li_children = ul.findChildren("li", recursive=False)
      for li in li_children:
         if type(li) == NavigableString:
            continue
         elif 'Top tourist spots:' in li.text:
            # Remove top tourist spots from the text and get the 2nd item which is 
            # texts with tourist destinations that are separated by comma
            cleaned_tourist_dest = li.text.split('Top tourist spots:')[1]
            # Split the text to get items as list
            cleaned_tourist_dest = cleaned_tourist_dest.split(',')
            # Get only the data that comes before a parentheses
            # Get the lower case 
            # Remove 'xa0' character using space
            cleaned_tourist_dest = [dest.split('(')[0]
                                    .lower()
                                    .strip()
                                    .replace(u'\xa0',' ')
                                    for dest in cleaned_tourist_dest]
            # Remove empty strings from the list
            cleaned_tourist_dest = list(filter(None, cleaned_tourist_dest))
            # Finally append values to the tourist_dest_list
            tourist_dest_list = tourist_dest_list + cleaned_tourist_dest
   print('DONE SCRAPING...')
   return tourist_dest_list

In [ ]:
from pathlib import Path

def create_tourist_dest_file():
   path = Path('../res/raw/tourist_dest.txt')
   if not path.is_file():
      tourist_dest_list = scrape_tourist_dest_list()
      tourist_dest_file = open('../res/raw/tourist_dest.txt', "w")
      for dest in tourist_dest_list:
         tourist_dest_file.write(f'{dest}\n')
   print('DONE CREATING TOURIST DEST FILE')


create_tourist_dest_file()


### Get Surnames

In [ ]:
import requests

url = 'https://baguiocityguide.com/how-common-is-your-last-name-here-are-the-top-1000-filipino-surnames'

header = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)
dfs = pd.read_html(r.text)

surnames_list = []
for table in dfs:
    surnames = [surname.lower() for surname in table['Surname'].to_list()]
    surnames_list = surnames_list + surnames
